In [2]:
!pip install opencv-python==4.5.5.64

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.7.0.72
    Uninstalling opencv-python-4.7.0.72:
      Successfully uninstalled opencv-python-4.7.0.72


In [3]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [ ]:
# import the necessary packages
from loader_util.preprocessing import ImageToArrayPreprocessor
from loader_util.preprocessing import SimplePreprocessor
from loader_util.preprocessing import PatchPreprocessor
from loader_util.preprocessing import MeanPreprocessor
from loader_util.callbacks import TrainingMonitor
from loader_util.io import HDF5DatasetGenerator
from loader_util.nn.conv import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import json
import os

In [ ]:
# script constants
mean_json_path = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chapter10_alexnet_dogs_cats/mean_dog_vs_cat.json"
train_dataset_path = r"/content/drive/MyDrive/Colab Notebooks/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/train.hdf5"
valid_dataset_path = r"/content/drive/MyDrive/Colab Notebooks/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/val.hdf5"
batch_size = 128
num_epochs = 100
output_path = r"/content/drive/MyDrive/Colab Notebooks/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/output"
model_output_path = os.path.sep.join([output_path, "saved_model.h5"])

In [ ]:
# construct the data augmentor
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
# load the RGB means for the training set
means = json.loads(open(mean_json_path).read())

# initialize the image preprocessors
sp = SimplePreprocessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [ ]:
# init the train and valis dataset generators
train_gen = HDF5DatasetGenerator(dbpath=train_dataset_path,
                                 batch_size=batch_size,
                                 aug=aug,
                                 preprocessors=[pp, mp, iap])

val_gen = HDF5DatasetGenerator(dbpath=valid_dataset_path,
                               batch_size=batch_size,
                               preprocessors=[sp, mp, iap])

[INFO] initialised HDF5 generator with 20000 data across 2 classes
[INFO] initialised HDF5 generator with 2500 data across 2 classes


In [ ]:
# initialize the optimizer
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = AlexNet.build(width=227, height=227, depth=3, classes=2, reg=0.0002)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# construct the set of callbacks
path = os.path.sep.join([output_path, "{}.png".format(os.getpid())])
callbacks = [TrainingMonitor(path)]
model.summary()

[INFO] compiling model...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 57, 57, 96)        34944     
                                                                 
 activation (Activation)     (None, 57, 57, 96)        0         
                                                                 
 batch_normalization (BatchN  (None, 57, 57, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 28, 96)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 28, 28, 96)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 256)       6

In [ ]:
# train the network
H = model.fit_generator(train_gen.generator(),
                        steps_per_epoch=train_gen.num_images // batch_size,
                        validation_data=val_gen.generator(),
                        validation_steps=val_gen.num_images // batch_size,
                        epochs=num_epochs,
                        max_queue_size=10,
                        callbacks=callbacks,
                        verbose=1)

<ipython-input-11-f65f5881d3ac>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(train_gen.generator(),


Epoch 1/100
156/156 [==============================] - 390s 2s/step - loss: 3.2089 - accuracy: 0.5929 - val_loss: 2.7048 - val_accuracy: 0.6427
Epoch 2/100
156/156 [==============================] - 405s 3s/step - loss: 2.3507 - accuracy: 0.6552 - val_loss: 1.9746 - val_accuracy: 0.6900
Epoch 3/100
156/156 [==============================] - 403s 3s/step - loss: 1.8158 - accuracy: 0.6861 - val_loss: 1.5992 - val_accuracy: 0.6727
Epoch 4/100
156/156 [==============================] - 406s 3s/step - loss: 1.4539 - accuracy: 0.7150 - val_loss: 1.2303 - val_accuracy: 0.7521
Epoch 5/100
156/156 [==============================] - 404s 3s/step - loss: 1.1830 - accuracy: 0.7340 - val_loss: 1.0452 - val_accuracy: 0.7549
Epoch 6/100
156/156 [==============================] - 405s 3s/step - loss: 1.0078 - accuracy: 0.7603 - val_loss: 0.8725 - val_accuracy: 0.8059
Epoch 7/100
156/156 [==============================] - 413s 3s/step - loss: 0.9129 - accuracy: 0.7837 - val_loss: 0.8550 - val_accuracy:

UnknownError: ignored

In [ ]:
# save the model to file
print("[INFO] serializing model...")
model.save(model_output_path, overwrite=True)
# close the HDF5 datasets
train_gen.close()
val_gen.close()

### evaluating with crop accuracy

In [7]:
!python "/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chapter10_alexnet_dogs_cats/2_eval_alexnet_crop_acc.py"

2023-06-07 07:10:55.969763: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 07:10:56.890839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[INFO] modeling model......
2023-06-07 07:11:05.671068: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-07 07:11:06.267059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at leas